# AoC - Day 7

## Part 1 - Problem Statement

Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of amplifiers already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.

```
    O-------O  O-------O  O-------O  O-------O  O-------O
    
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)

    O-------O  O-------O  O-------O  O-------O  O-------O
```

The Elves have sent you some Amplifier Controller Software (your puzzle input), a program that should run on your existing Intcode computer. Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current phase setting (an integer from 0 to 4). Each phase setting is used exactly once, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to find the largest output signal that can be sent to the thrusters by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.

For example, suppose you want to try the phase setting sequence 3,1,2,4,0, which would mean setting amplifier A to phase setting 3, amplifier B to setting 1, C to 2, D to 4, and E to 0. Then, you could determine the output signal that gets sent from amplifier E to the thrusters with the following steps:

* Start the copy of the amplifier controller software that will run on amplifier A. At its first input instruction, provide it the amplifier's phase setting, 3. At its second input instruction, provide it the input signal, 0. After some calculations, it will use an output instruction to indicate the amplifier's output signal.
* Start the software for amplifier B. Provide it the phase setting (1) and then whatever output signal was produced from amplifier A. It will then produce a new output signal destined for amplifier C.
* Start the software for amplifier C, provide the phase setting (2) and the value from amplifier B, then collect its output signal.
* Run amplifier D's software, provide the phase setting (4) and input value, and collect its output signal.
* Run amplifier E's software, provide the phase setting (0) and input value, and collect its output signal.

The final output signal from amplifier E would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.

## Solution

In [24]:
from functools import reduce
from itertools import permutations
import operator

In [2]:
def read_program(program):
    return [int(token) for token in program.split(',')]

In [3]:
def build_memory(program):
    return dict(enumerate(read_program(program)))

In [4]:
operation = {
    '01': 'add',
    '02': 'multiply',
    '03': 'input',
    '04': 'output',
    '05': 'jump-if-true',
    '06': 'jump-if-false',
    '07': 'less-than',
    '08': 'equals'
}

In [5]:
mode = {
    '0': 'position',
    '1': 'immediate'
}

In [6]:
def parse_opcode(opcode):
    opcode = f'{opcode:05}'
    instruction = opcode[3:]
    
    op = operation[instruction]
    if op in ('add', 'multiply', 'less-than', 'equals'):
        return {
            'operation': op,
            'mode_param1': mode[opcode[2]],
            'mode_param2': mode[opcode[1]],
            'mode_param3': 'target',
            'ip_shift': 4
        }
    elif op in ('input', 'output'):
        return {
            'operation': op,
            'mode_param1': 'target' if mode[opcode[2]] != 'immediate' else 'immediate',
            'ip_shift': 2
        }
    elif op in ('jump-if-true', 'jump-if-false'):
        return {
            'operation': op,
            'mode_param1': mode[opcode[2]],
            'mode_param2': mode[opcode[1]],
            'ip_shift': 'jump'
        }
    else:
        assert False

In [7]:
def get_param_value(memory, mode, value):
    return memory[value] if mode == 'position' else value

In [8]:
def parse_instruction(ip, memory):
    opcode = memory[ip]
    instruction = parse_opcode(opcode)
    operation = instruction['operation']
    
    if operation in ('add', 'multiply', 'less-than', 'equals'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        param2 = get_param_value(memory, instruction['mode_param2'], memory[ip+2])
        param3 = get_param_value(memory, instruction['mode_param3'], memory[ip+3])
        return {'operation': operation,
                'param1': param1,
                'param2': param2,
                'param3': param3,
                'next_ip': ip + instruction['ip_shift']}
    elif operation in ('input', 'output'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        return {'operation': operation,
                'param1': param1,
                'mode_param1': instruction['mode_param1'],
                'next_ip': ip + instruction['ip_shift']}
    elif operation in ('jump-if-true', 'jump-if-false'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        param2 = get_param_value(memory, instruction['mode_param2'], memory[ip+2])
        return {'operation': operation,
                'param1': param1,
                'param2': param2,
                'next_ip': 'jump'}
    else:
        assert False

In [9]:
def execute_instruction(ip, memory, input_generator, output):
    instruction = parse_instruction(ip, memory)
    
    if instruction['operation'] == 'add':
        memory[instruction['param3']] = instruction['param1'] + instruction['param2']
    elif instruction['operation'] == 'multiply':
        memory[instruction['param3']] = instruction['param1'] * instruction['param2']
    elif instruction['operation'] == 'input':
        memory[instruction['param1']] = int(next(input_generator))
    elif instruction['operation'] == 'output':
        output.append(instruction['param1'] if instruction['mode_param1'] == 'immediate' else memory[instruction['param1']])
    elif instruction['operation'] == 'jump-if-true':
        instruction['next_ip'] = instruction['param2'] if instruction['param1'] != 0 else ip + 3
    elif instruction['operation'] == 'jump-if-false':
        instruction['next_ip'] = instruction['param2'] if instruction['param1'] == 0 else ip + 3
    elif instruction['operation'] == 'less-than':
        memory[instruction['param3']] = 1 if instruction['param1'] < instruction['param2'] else 0
    elif instruction['operation'] == 'equals':
        memory[instruction['param3']] = 1 if instruction['param1'] == instruction['param2'] else 0
    else:
        assert False
    
    return {
        'ip': instruction['next_ip'],
        'memory': memory,
        'output': output
    }

In [10]:
def run_program(program, input_generator):
    memory = build_memory(program)
    ip = 0
    
    output = []
    while memory[ip] != 99:
        ip, memory, output = operator.itemgetter('ip', 'memory', 'output')(execute_instruction(ip, memory, input_generator, output))
    
    return output

In [11]:
def simulate_amplifier_chain(program, settings):
    
    input_0 = 0
    output_1 = run_program(program, (i for i in (settings[0], input_0)))[0]
    output_2 = run_program(program, (i for i in (settings[1], output_1)))[0]
    output_3 = run_program(program, (i for i in (settings[2], output_2)))[0]
    output_4 = run_program(program, (i for i in (settings[3], output_3)))[0]
    output_5 = run_program(program, (i for i in (settings[4], output_4)))[0]

    return output_5

## Tests

In [12]:
assert run_program('3,9,8,9,10,9,4,9,99,-1,8', (i for i in [8])) == [1]
assert run_program('3,9,8,9,10,9,4,9,99,-1,8', (i for i in [7])) == [0]

In [13]:
assert run_program('3,9,7,9,10,9,4,9,99,-1,8', (i for i in [8])) == [0]
assert run_program('3,9,7,9,10,9,4,9,99,-1,8', (i for i in [7])) == [1]

In [14]:
assert run_program('3,3,1108,-1,8,3,4,3,99', (i for i in [8])) == [1]
assert run_program('3,3,1108,-1,8,3,4,3,99', (i for i in [7])) == [0]

In [15]:
assert run_program('3,3,1107,-1,8,3,4,3,99', (i for i in [8])) == [0]
assert run_program('3,3,1107,-1,8,3,4,3,99', (i for i in [7])) == [1]

In [16]:
assert run_program('3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9', (i for i in [0])) == [0]
assert run_program('3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9', (i for i in [10])) == [1]

In [17]:
assert run_program('3,3,1105,-1,9,1101,0,0,12,4,12,99,1', (i for i in [0])) == [0]
assert run_program('3,3,1105,-1,9,1101,0,0,12,4,12,99,1', (i for i in [10])) == [1]

In [18]:
test_program = '''
3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99'''

assert run_program(test_program, (i for i in [7])) == [999]
assert run_program(test_program, (i for i in [8])) == [1000]
assert run_program(test_program, (i for i in [9])) == [1001]

In [19]:
test_program = '3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0'
assert simulate_amplifier_chain(test_program, (4,3,2,1,0)) == 43210

In [20]:
test_program = '3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0'
assert simulate_amplifier_chain(test_program, (0,1,2,3,4)) == 54321

In [21]:
test_program = '3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0'
assert simulate_amplifier_chain(test_program, (1,0,4,3,2)) == 65210

## Data

In [22]:
program = '3,8,1001,8,10,8,105,1,0,0,21,38,55,64,81,106,187,268,349,430,99999,3,9,101,2,9,9,1002,9,2,9,101,5,9,9,4,9,99,3,9,102,2,9,9,101,3,9,9,1002,9,4,9,4,9,99,3,9,102,2,9,9,4,9,99,3,9,1002,9,5,9,1001,9,4,9,102,4,9,9,4,9,99,3,9,102,2,9,9,1001,9,5,9,102,3,9,9,1001,9,4,9,102,5,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,99,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,99'

In [25]:
signals = []
signals = [simulate_amplifier_chain(program, settings)
           for settings in permutations(range(5))]
max(signals)   

117312

## Part 2 - Problem Statement

## Solution

## Tests